![](./img/chinahadoop.png)
# 风控实战项目 -- 电信行业风控建模
**[小象学院](http://www.chinahadoop.cn/course/landpage/15)《机器学习集训营》实战项目案例 by [@寒小阳](http://www.chinahadoop.cn/user/49339/about)**

## 说明
本notebook为特征工程部分，给出了基本的思路和框架，请大家填充完相应环节的步骤，同时也鼓励大家尝试更多更好的数据处理与特征工程和建模方式。

### 引入工具库

In [2]:
import numpy as np
import pandas as pd
import scipy as sp
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, f1_score
import xgboost as xgb

咱们从单表和多表交叉，分别抽取单维度特征和交叉维度特征

### sms表特征抽取
#### 先对所有的时间进行特征提取
1. 用户电话的总的通话次数 opp_num
2. 通话的人数，sms_all_unique_cnt
3. 通话次数 / 人数的比例， sms_all_cnt_all_unique_cnt_rate
4. 对端电话的前n位的个数，所有的不同号码的个数。 opp_head
5. 对端号码长度的分布个数   opp_len
6. 通话最大时长，平均时长，最小时长，极差时长等统计的信息  start_time
7. 通话类型的分布个数或者比例 call_type
8. 通话类型的分布个数和比例   in_out

In [3]:
# 读取数据
df_train_sms = pd.read_csv('./data/sms_train.txt', sep='\t', names= ['uid','opp_num','opp_head','opp_len','start_time','in_out'], low_memory=False)
df_train_label = pd.read_csv('./data/uid_train.txt', sep='\t', names = ['uid', 'label'], low_memory=False)

df_testA_sms = pd.read_csv('./data/sms_test_a.txt', sep='\t', names= ['uid','opp_num','opp_head','opp_len','start_time','in_out'], low_memory=False)
df_testB_sms = pd.read_csv('./data/sms_test_b.txt', sep='\t', names = ['uid', 'label'], low_memory=False)

In [ ]:
def get_sms_features(df_sms, mode='train', Type=None):
    # 为了保证函数通用性，我们设定mode来标示训练集和不同的测试集处理
    if mode == 'train':
        # 复制lable的数据
        df_train = df_train_label.copy()
    else:
        if Type=='A':
            df_train = pd.DataFrame(data={'uid':['u'+str(id) for id in range(5000, 7000)]})
        else:
            df_train = pd.DataFrame(data={'uid':['u'+str(id) for id in range(7000, 10000)]})
    
    # 特征：年月日
    # 提示：to_datetime的应用
    # your code here
    
    
    # 特征：user总通话次数
    # 提示：分组统计
    # your code here
    
    
    # 特征：总的通话的对端的不重复的个数，即opp_num不同的个数
    # 提示：分组统计
    # your code here
    
    
    # 特征：平均每个对端(人)通话的次数， 即 sms_all_per_opp_rate = 通话次数 / 人数
    # 提示：两列除法，比例
    # your code here
    
    
    # 特征：对端电话的前n位的个数，所有的不同号码的个数以及其所有的分布个数和比例(部分特征待定)。 opp_head_cnt_{k}, opp_head_rate_{k}
    # 提示：分组统计
    # your code here
    
    
    
    # 联系最多和最少的次数的opp_head
    # 提示：分组统计
    # your code here
    
    
    # 联系最多的head的个数
    # your code here
    
    
    # 平均联系的head的个数
    # your code here
    
    
    
    # 最近一次联系的号码的长度
    # your code here
    

    
    # 联系最大时长，平均时长，最小时长，极差时长等统计的信息
    # your code here
    
    
    # 联系的密度，平均多长时间联系1次
    # your code here
    
    
    # day的分布，和比例
    # 也就是一个月中每一天(day)的数量分布占比
    # your code here
    
    
    
    # hour分布和比例
    # 也就是一个天中小时(hour)的数量分布占比
    # your code here
    
    
    
    # minute 分布
    # your code here
    

    
    # 平均联系时间间隔
    # your code here
    
    
    # 白天和晚上(或者一天中不同时间段)的联系频度和间隔
    # your code here
    
    
    # 其他特征
    # your code here
    

                      
    return df_train

In [ ]:
# 功能函数：时间差，计算两个时间点秒数的差别
def diff_time(a,b):
    a_day, a_hour, a_minute, a_second = (a / 1000000, a / 10000 % 100, a / 100 % 100, a % 100)
    b_day, b_hour, b_minute, b_second = (b / 1000000, b / 10000 % 100, b / 100 % 100, b % 100)
    
    d_day = b_day - a_day
    d_hour = b_hour - a_hour
    d_minute = b_minute - a_minute
    d_second = b_second - a_second
    
    diff = d_day * 24 * 60 * 60 + d_hour * 60 * 60 + d_minute * 60 + d_second
    return diff

def get_diff_time(x):
    diff_t = []
    for d in x:
        diff_t.append(diff_time(d[0],d[1]))
    return diff_t

### 调用特征工程处理

In [ ]:
df_train = get_sms_features(df_train_sms)
df_testA = get_sms_features(df_testA_sms, mode='test')
df_test = get_sms_features(df_testB_sms, mode='test')

# 缺失值填充
df_train.fillna(0,inplace=True)
df_test.fillna(0,inplace=True)

# 中间特征存储
df_train.to_csv('./features/df_train_sms_feat.csv',index=False)
df_test.to_csv('./features/df_testB_sms_feat.csv',index=False)
df_testA.to_csv('./features/df_testA_sms_feat.csv',index=False)

### 查看数据

In [ ]:
df_train.info()
df_test.info()
df_test.head()